<div>
  <div>
    Emme Notebook and Scripting Course, August 2019 <br>
  </div>
  <div>
    <img style="align: left; margin: 15px 15px 15px 0px;" src="./INRO Logo.png" width="120" />
  </div>
  <div>
    © Copyright 2019 INRO
  </div>
</div>

# 4 Desktop API - Automate Charts and Reports

This lesson shows how the Desktop API can be used to automate Emme Desktop, including:
- Opening, creating and customizing maps, charts and tables
- Exporting images and files 
- Changing the view 
- Changing the current Database and Scenarios 
To learn further, refer to the Desktop API Guide and Emme API Reference. 

__Suggested duration :__ 2 hours

## 4.1 Contents

<a href="#4.1-Contents">4.1 Contents</a>

<a href="#4.2-Example-1---Export-images-for-turning-movement-results-for-all-intersection-and-all-scenarios">4.2 Example 1 - Export images for turning movement results for all intersection and all scenarios</a>

- <a href="#4.2.1-Accessing-an-existing-worksheet-item">4.2.1 Accessing an existing worksheet item</a>
- <a href="#4.2.2-Opening-a-worksheet">4.2.2 Opening a worksheet</a>
- <a href="#4.2.3-Modifying-an-existing-worksheet">4.2.3 Modifying an existing worksheet</a>
- <a href="#4.2.4-Setting-the-view">4.2.4-Setting the view</a>
- <a href="#4.2.5-Exporting-a-worksheet-as-image">4.2.5 Exporting a worksheet as image</a>
- <a href="#4.2.6-Accessing-scenarios">4.2.6 Accessing scenarios</a>
- <a href="#4.2.7-Iterating-through-intersections">4.2.7 Iterating through intersections</a>
- <a href="#4.2.8-PRACTICE:-Desktop-API">4.2.8 PRACTICE: Desktop API</a>
- <a href="#4.2.9-Additional-Example---Extract-as-image-the-worksheet-for-each-scenario-and-each-intersection">4.2.9 Additional example - Extract as image the worksheet for each scenario and each intersection</a>

<a href="#4.3-Example-2---Extracting-transit-volumes-for-each-transit-line-in-each-scenario">4.3 Example 2 - Extracting transit volumes for each transit line in each scenario</a>
- <a href="#4.3.1-Modify-Network-Table-Parameters">4.3.1 Modify Network Table Parameters</a>
- <a href="#4.3.2-Export-a-network-table-as-CSV">4.3.2 Export a network table as CSV</a>
- <a href="#4.3.3-Additional-Example---Export-the-Network-Table-for-each-transit-line-and-each-scenario-to-CSV-file">4.3.3 Additional Example - Export the Network Table for each transit line and each scenario to CSV file</a>

## 4.2 Example 1 - Export worksheet for turning movement results for all intersections and all scenarios
In this example we will load the worksheet _Volumes (at intersections)_ and iterate through scenario 1000, 2000 and 3000 and. For each scenario we will export the worksheet as image for each intersection. We will learn the following skills:

- access and open a worksheet
- modify a worksheet
- export a worksheet as image
- iterate through the scenarios
- access the scenario and iterate through its nodes



The entry point for the Desktop API is the <code><b>inro.emme.desktop.app.App</b></code> object, which represents an application of the Emme Desktop.

When scripting outside of the Emme Notebook, there are two ways to create an <code>App</code> object: by connecting to an already running application of the Emme desktop software or by starting a new instance of Emme and then connecting to it.

In an Emme Notebook, the connection is done automatically. Let's create our <code>App</code> object.

In [ ]:
desktop = inro.modeller.Modeller().desktop

From this object, we can now access the project properties, the Explorer pane, the worksheets and tables, etc.

Many common uses of the Desktop API involve customizing and/or printing worksheets. The relevant classes are those available in the <code><b>worksheet</b></code> module, such as: <code><b>WorksheetFolder</b></code>, <code><b>WorksheetItem</b></code> and <code><b>Worksheet</b></code>.

### 4.2.1 Accessing an existing worksheet item

In order to open a specific worksheet, we need to find it in its hierarchy of folders. The easiest way to do this is to use the `ExplorerFolder.find_item()` method, or the `ExplorerFolder.find_items()` method (to return multiple worksheets in the same path). First, let's get the root_worksheet_folder.

In [ ]:
root_worksheet_folder = desktop.root_worksheet_folder()

To access a specific worksheet which path is known, the `find_item()` method lets you access it directly.

In [ ]:
worksheet_path = ["General", "Results Analysis", "Traffic", "Volumes (at intersections)"]
intersection_worksheet_item = root_worksheet_folder.find_item(worksheet_path)
intersection_worksheet_item.name()

__Tip__ If the path is not known, it is also possible to parse through all folders and look for the worksheet using the method `all_subfolders()` and looking through all its children.

In [ ]:
worksheet_name = "Volumes (at intersections)"
found = False
for folder in root_worksheet_folder.all_subfolders():
    print "... searching in folder", folder.name()
    for item in folder.child_items():
        if item.name() == worksheet_name:
            print "    Worksheet found!", item.name()
            intersection_worksheet_item = item
            found = True
    if found:
        break

### 4.2.2 Opening a worksheet
A worksheet can be opened from a worksheet item by using `worksheet.open()` method.

In [ ]:
intersection_worksheet = intersection_worksheet_item.open()

### 4.2.3 Modifying an existing worksheet
Let's display the legend. The legend layer can be accessed using the `worksheet.layer(layer_type, layer_name)` method. The layer name of our legend is _Legend_.

In [ ]:
legend_layer = intersection_worksheet.layer(
    layer_type='Legend',
    layer_name='Legend'
)

Layer parameters can be accessed using the `layer.par(name)` method. All available parameters are documented in the Emme Help under _Emme Desktop Manual -> Parameters -> Layer Parameters_. You can also right-click on any parameter in the worksheet layer control panel in Emme Desktop to see its name.

Let's make the layer visible by setting its show flag _SFlag_ to `True`.

In [ ]:
legend_layer.par('SFlag').set(True)

The displayed intersection can be modified using the intersection filter parameter _Filter0_ in the _Intersection_ layer. Let's get the intersection layer and change it's filter to node 204.

In [ ]:
intersection_layer = intersection_worksheet.layer(
    layer_type='Configurable control',
    layer_name='Intersection'
)
intersection_layer.par('Filter0').set('i==204')

### 4.2.4 Setting the view
Views are accessed in a similar manner as accessing a worksheet. We first access the view folder from the `desktop` object, and then use the `ExplorerFolder.find_item()` method to access the view. Let's fetch the _Downtown_ view.

In [ ]:
root_view_folder = desktop.root_view_folder()
view_path = ['Districts', 'Downtown']
downtown_view = root_view_folder.find_item(view_path)

Note that views can be returned as a `Box` object. The `Box` object is a bounding box containing starting (`x1`,`y1`) and ending (`x2`,`y2`) coordinates to define the rectangle corresponding to this view.

In [ ]:
downtown_box = downtown_view.get_box()
print "Box start coordinates:", downtown_box.x1, downtown_box.y1
print "Box end coordinates:", downtown_box.x2, downtown_box.y2

The next step is to set the view for our worksheet. Let's open the General Worksheet and set the view of the worksheet to _Downtown_. We will use the `worksheet.set_view` method, which accepts an argument containing the `Box` of the _Downtown_ view.

In [ ]:
general_worksheet_path = ["General", "General worksheet"]
general_worksheet_item = root_worksheet_folder.find_item(general_worksheet_path)
general_worksheet = general_worksheet_item.open()

general_worksheet.set_view(downtown_box)

### 4.2.5 Exporting a worksheet as image
The Desktop API lets you export worksheets as an image, SVG or PDF using one of the following methods:

- `worksheet.save_as_image()`
- `worksheet.save_as_svg()`
- `worksheet.save_as_pdf()`

Extensive information for each of these method can be found in the documentation. Note that these methods only accept absolute paths. The code below saves our previously opened intersection worksheet as an image (PNG file) using the default settings:

In [ ]:
import os
project_directory = os.path.dirname(desktop.project.path)
image_filepath = os.path.join(project_directory, 'Logbook', 'intersection_204.png')
intersection_worksheet.save_as_image(image_filepath, size=(800, 600))

The other parameters are optional. They can be used to specify:
- a quality factor, in percent, which represents compression (100% is highest quality);
- a detail factor which will increase the amount of detail shown up to a maximum factor of 8 (useful for printing plots larger than the screen view); 
- margins; margins are set on a <code>Margins</code> object from the types module, which is passed to the <code><b>save_as_image()</b></code> (or <code><b>save_as_svg()</b></code> method, more below). Note that margins are expressed as a percentage for <code><b>save_as_image()</b></code>;
- a view box, as an instance of <code>Box</code>, which defines a specific view instead of the default current view.

Note that higher size, detail and quality factors will result in larger file sizes. Also, be careful increasing the Detail when using a Web basemap layer, as this can cause extremely big tile downloads especially on higher-resolution monitors.

Let's try a modified version of the previous example to see the difference.

In [ ]:
import inro.emme.desktop.types as _types

my_margins = _types.Margins()
my_margins.left = 5.0
my_margins.top = 5.0
my_margins.right = 5.0
my_margins.bottom = 5.0

image_filepath = os.path.join(project_directory, 'Logbook', 'intersection_204(2).png')

intersection_worksheet.save_as_image(
    image_filepath,
    size=(800, 600),
    quality=81,
    detail=1,
    margins=my_margins
)

### 4.2.6 Accessing scenarios
The main entry point to manipulate scenarios in the Desktop API is an object of class <code><b>DataExplorer</b></code>. This object is available directly from the Desktop <code>App</code> object using <code><b>data_explorer()</b></code>:

In [ ]:
data_explorer = desktop.data_explorer()

The <i>data_explorer</i> represents the available data in the Explorer window - the collection of databases and scenarios in the project, as well as the Configurable Attributes. We can use it to access the <code>Database</code> object(s), which we can in turn use to access the <code>Scenario</code> objects contained in the database.

The <code>DataExplorer</code> also allows us to directly access some key status information, such as the list of the current <i>Active Scenarios</i>. Here, we start by printing the list of active scenarios using the <code>DataExplorer<b>.active_scenarios()</b></code> method.

In [ ]:
for scenario in data_explorer.active_scenarios():
    print "Active scenario: ", scenario.number(), scenario.title()

The <code><b>active_scenarios()</b></code> method returns a list of <code>Scenario</code> objects, and we can call methods on this objects to obtain information about the scenario.

It is also possible to display the title of the <i>Active Database</i> and to list all the scenarios in the database using the <code>DataExplorer<b>.active_database()</b></code> and <code>Database<b>.scenarios()</b></code> methods.

In [ ]:
database = data_explorer.active_database()
print database.title()

for scenario in database.scenarios():
    print scenario.title()

Use the method `database.scenario_by_number(scenario_id)` to access a specific scenario from the database. Then use `data_explorer.replace_primary_scenario(scenario)` to make the selected scenario the primary scenario.

In [ ]:
scenario_1000 = database.scenario_by_number(1000)
data_explorer.replace_primary_scenario(scenario_1000)

### 4.2.7 Iterating through intersections
To be able to iterate through the intersections we can use the Network API to iterate through the nodes and check whether or not they are intersections. 

It is important to note the difference bewteen:

- the Desktop API scenario - `class inro.emme.desktop.data.Scenario` 
- the Network API scenario `class inro.emme.database.scenario.Scenario`. 

To access the Network API scenario from a Scenario in Emme Desktop, use the `core_scenario` parameter. 

Ex: `database_scenario = desktop_scenario.core_scenario` 




We will first retrieve the network, and then iterate through it to display the nodes which are intersections.

In [ ]:
core_scenario = scenario_1000.core_scenario
network = core_scenario.get_network()
for node in network.nodes():
    if node.is_intersection:
        print node

### 4.2.8 <span style="color:red">PRACTICE: Desktop API</span>

Please refer to the _Emme Notebook and Scripting - Practices_ Notebook to complete this exercise. Note that the solutions to practices are found in the _Emme Notebook and Scripting - Solutions_ Notebook.

### 4.2.9 Additional Example - Extract as image the worksheet for each scenario and each intersection
Let's put together what we learned above and save our intersection images iteratively. We can process each intersection using `for` loops, and save the images into a specific directory using a naming convention for each file. We will save the images in the _Logbook_ directory, under a sub-folder called 'intersection_volumes'. The naming convention will be as follows:

'[NODE\_ID]\_[SCENARIO_ID]\_intersection\_volumes.png'

In [ ]:
project_directory = os.path.dirname(desktop.project.path)

The `os.mkdir(path)` function let's us create a directory as the `path` location.

In [ ]:
output_directory = os.path.join(project_directory, 'Logbook', 'intersection_volumes')
os.mkdir(output_directory)

In [ ]:
# Access the worksheet folder
root_worksheet_folder = desktop.root_worksheet_folder()

# Access the Volumes at intersections worksheet
worksheet_path = ["General", "Results Analysis", "Traffic", "Volumes (at intersections)"]
intersection_worksheet_item = root_worksheet_folder.find_item(worksheet_path)
intersection_worksheet = intersection_worksheet_item.open()

# Show the legend and set the text size
legend_layer = intersection_worksheet.layer(
    layer_type='Legend',
    layer_name='Legend'
)
legend_layer.par('SFlag').set(True)
legend_layer.par('TextSize').set(20.)

# Access the intersection layer and set the text size to 12
intersection_layer = intersection_worksheet.layer(
    layer_type='Configurable control',
    layer_name='Intersection'
)
intersection_layer.par('Float2').set(12.)


# Set up the margins for saving as image
my_margins = _types.Margins()
my_margins.left = 5.0
my_margins.top = 5.0
my_margins.right = 5.0
my_margins.bottom = 5.0

# Iterate through the scenarios
for scenario_id in [1000, 2000, 3000]:
    # Make the current scenario the primary scenario
    scenario = database.scenario_by_number(scenario_id)
    data_explorer.replace_primary_scenario(scenario)
    
    # Access the network of the current scenario
    core_scenario = scenario.core_scenario
    network = core_scenario.get_network()
    
    # Iterate through the scenario's nodes
    for node in network.nodes():
        # If the node is an intersection, set the intersection layer filter and extract an image
        if node.is_intersection:
            # Change the layer's filter to display the intersection
            intersection_layer.par('Filter0').set('i==%i' % node.number)
            
            # save as image at the specified location
            image_filepath = os.path.join(
                output_directory, 
                '%s_%s_intersection_volume.png' % (node.id, scenario.number())
            )
            
            intersection_worksheet.save_as_image(
                image_filepath,
                size=(1000, 1000),
                quality=100,
                detail=1,
                margins=my_margins
            )
            
# Close the worksheet
intersection_worksheet.close()         
    
    

## 4.3 Example 2 - Extracting transit volumes tables for each transit line in each scenario

Now let's work with tables in a similar way. We will load the Network Table _Segment results/by line_ and iterate through scenarios 1000, 2000 and 3000 to extract the transit volume information for each transit line and save this data. We will learn the new following skills:

- modify a network table
- export a network table as CSV

We can use what we already know about looking up a worksheet item to find our table: 

In [ ]:
desktop = inro.modeller.Modeller().desktop
root_worksheet_folder = desktop.root_worksheet_folder()
worksheet_path = ["General", "Results Analysis", "Transit", "Segment results/by line"]
segment_results_item = root_worksheet_folder.find_item(worksheet_path)
segment_results = segment_results_item.open()

### 4.3.1 Modify Network Table Parameters
As with <code>Worksheets</code> and <code>Layers</code>, we can use the <code><b>.par()</b></code> method to access and change <code>Table</code> <code><b>Parameters</b></code>. Let's change the filter parameter:

In [ ]:
segment_results.par('ByFilter').set('line=="15ae"')

### 4.3.2 Export a network table as CSV
Network tables can be printed using the <code>NetworkTable<b>.print_table()</b></code> method and exported to a text file using the <code>NetworkTable<b>.export()</b></code> method as shown below.

In [ ]:
export_filepath = os.path.join(project_directory, 'Logbook', 'segment_results_15ae.txt')
segment_results.export(export_filepath)

We can optionnaly modify the column separator (which is a Network Table parameter) to use a comma. 

In [ ]:
segment_results.par('ExportColumnSeparator').set(',')
export_filepath = os.path.join(project_directory, 'Logbook', 'segment_results_15ae.txt')
segment_results.export(export_filepath)

### 4.3.3 Additional Example - Export the Network Table for each transit line and each scenario to CSV file

Use what we have already learned to save transit segment results by transit line and by scenario (1000, 2000, 3000).

In [ ]:
output_directory = os.path.join(project_directory, 'Logbook', 'transit_line_volumes')
os.mkdir(output_directory)

In [ ]:
# access the segment results table
desktop = inro.modeller.Modeller().desktop
root_worksheet_folder = desktop.root_worksheet_folder()
worksheet_path = ["General", "Results Analysis", "Transit", "Segment results/by line"]
segment_results_item = root_worksheet_folder.find_item(worksheet_path)
segment_results = segment_results_item.open()

# set its column separator to comma
segment_results.par('ExportColumnSeparator').set(',')

# Iterate through the scenarios
for scenario_id in [1000, 2000, 3000]:
    # Make the current scenario the primary scenario
    scenario = database.scenario_by_number(scenario_id)
    data_explorer.replace_primary_scenario(scenario)
    
    # Access the network of the current scenario
    core_scenario = scenario.core_scenario
    network = core_scenario.get_network()
    
    # Iterate throught the transit lines
    for transit_line in network.transit_lines():
        # Update the Network table filter
        segment_results.par('ByFilter').set('line=="%s"' % transit_line.id)
        
        # extract the network table
        table_filepath = os.path.join(
            output_directory,
            '%s_%s_segment_volumes.csv' % (core_scenario.id, transit_line.id)
        )
        
        segment_results.export(table_filepath)
        
segment_results.close()